In [1]:
URL = "chals.sekai.team:3062"
FLAG1 = "SEKAI{GES_15_34sy_2_br34k_kn@w1ng_th3_k3y}"

In [2]:
import utils
import socket
import sys
import time
import functools
from collections import defaultdict 

NODE_COUNT = 130
SECURITY_PARAMETER = 32

class CryptoClient:
    def __init__(self, connection: str):
        self.ip, self.port = connection.split(":")
        self.port = int(self.port)
        self.sock = None
        self.buffer = ""  # Initialize an empty buffer

    def connect(self):
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        try:
            self.sock.connect((self.ip, self.port))
        except ConnectionError:
            print("Failed to connect to server.")
            sys.exit(1)
        print("CONNECTED")

    def read_line(self):
        while "\n" not in self.buffer:
            try:
                data = self.sock.recv(1024).decode('utf-8')
                if not data:
                    return None
                self.buffer += data
            except socket.error:
                return None
        line, self.buffer = self.buffer.split("\n", 1)
        return line
    
    def skip_until(self, text, debug=True):
        time.sleep(1)
        while True:
            line = self.read_line()
            if debug and line:
                print(line)
            if line and text in line:
                break
                
    def receive_destination(self):
        line = self.read_line()
        if line:
            destination = int(line.split(':')[-1].strip())
        else:
            print("Failed to receive data from server.")
            sys.exit(1)
        print(f"DESTINATION: {destination}")
        return destination

    def receive_query(self):
        line = self.read_line()
        if line:
            values = line.split(': ')[-1].split()
            query_from, query_to = map(int, values)
        else:
            print("Failed to receive data from server.")
            sys.exit(1)
        print(f"QUERY: {query_from} -> {query_to}")
        return query_from, query_to

    def get_hex_value(self):
        line = self.read_line()
        if line:
            try:
                value = bytes.fromhex(line.split(': ')[-1].strip())
            except ValueError as e:
                print(line)
                raise
        else:
            print("Failed to receive data from server.")
        return value
    
    def receive_response(self, debug=False):
        response = self.get_hex_value()
        if debug:
            print(f"RESPONSE: {response.hex()}")    
        return response
    
    def receive_token(self, debug=False):
        token = self.get_hex_value()
        if debug:
            print(f"TOKEN: {token.hex()}")    
        return token
    
    def send_response(self, response: str, debug=False):
        try:
            self.sock.sendall((response + "\n").encode('utf-8'))
        except socket.error:
            print("Failed to send data to server.")
            sys.exit(1)
        if debug:
            print(f"SENT RESPONSE: {response}")


def decrypt(u: int, v: int, key: bytes, resp: bytes) -> str:
    """
    Decrypts the response to reveal the shortest path between nodes in the graph.

    Parameters:
    u (int): The source node
    v (int): The destination node
    resp (bytes): The encrypted response from the server
    key (bytes): The secret key used for decryption

    Returns:
    str: The decrypted shortest path as a space-separated string
    """
    # Extract Subkeys
    key_SKE = key[:16]
    key_DES = key[16:]
    
    # Initialize Variables
    path = [u]
    # Decryption Loop
    segment_size = 32  # Assuming each encrypted node is 32 bytes
    for i in range(0, len(resp), segment_size):
        # Decrypt the segment
        block = resp[i:i+segment_size]
        decrypted_segment = utils.SymmetricDecrypt(key_SKE, block)
        
        # Path Extraction
        # Assuming decrypted_segment contains the node as bytes
        node1, node2 = map(int, decrypted_segment.decode().split(','))
        path.append(node1)
        
    # Validate the path starts at 'u' and ends at 'v'
    if path[0] != u or path[-1] != v:
        return "Invalid path"
    
    # Return Result
    return " ".join(map(str, path))

    
debug = True
if __name__ == "__main__":
    client = CryptoClient(URL)
    client.connect()
    client.send_response(FLAG1)  # Uncomment this if FLAG1 is defined elsewhere
    client.skip_until('Pass 10 challenges to get the flag', debug=False)
    
    for _ in range(10):
        tokens = []
        search_tokens = []
        responses = []

        path_from = {}
        ids = {}
        def get_id(s):
            ids[s] = ids.get(s, len(ids))
            return ids[s]

        SIZE = SECURITY_PARAMETER * 2
        def enumerate_unique(string):
            numbers = []
            for i in range(len(string) // SIZE):
                chunk = string[i * SIZE: (i + 1) * SIZE]
                numbers.append(get_id(chunk))

            return numbers

        def str_enumerate_unique(numbers):
            return ','.join([str(n) for n in numbers])
        client.skip_until('Encrypting graph', debug=True)
        destination = client.receive_destination()
        
        # Prepare an empty bytearray to collect all responses
        all_responses = bytearray()
        
        for i in range(NODE_COUNT):
            if i == destination:
                query_from, query_to = 0, 1
            else:
                query_from, query_to = i, destination
            query = f'{query_from},{query_to}'
            client.send_response(query)
            token = client.receive_token()
            ids[token.hex()] = query_from
            search_token_response = client.receive_response()
            response_len = len(search_token_response)
            search_token = search_token_response[:response_len // 2]
            response = search_token_response[response_len // 2:]
            if query_to == destination:
                path_from[query_from] = search_token
            
            tokens.append(token)
            search_tokens.append(search_token)
            responses.append(response)
            
        graph = defaultdict(set)
        print("Graph shortest pathes:")
        for u, path in path_from.items():
            full_path = [u] + enumerate_unique(path.hex())[:-1] + [destination]
            if debug:
                output = ' -> '.join([
                    str(node) for node in full_path
                ])
                print(output)
            
            for a, b in zip(full_path, full_path[1:]):
                graph[a].add(b)
                graph[b].add(a)
                
        SDSP_node_degrees = []
        for node, neighbours in graph.items():
            SDSP_node_degrees.append(len(neighbours))
            
        SDSP_node_degrees = list(sorted(SDSP_node_degrees)) 
        client.send_response(' '.join([str(n) for n in SDSP_node_degrees]), debug=True)
                    
    client.skip_until("Flag:")

CONNECTED
[+] Challenge 1/10. Generating random graph...
[+] Encrypting graph...
DESTINATION: 7
Graph shortest pathes:
0 -> 24 -> 39 -> 7
1 -> 3 -> 62 -> 18 -> 7
2 -> 41 -> 123 -> 7
3 -> 62 -> 18 -> 7
4 -> 52 -> 49 -> 31 -> 18 -> 7
5 -> 82 -> 24 -> 39 -> 7
6 -> 125 -> 39 -> 7
8 -> 60 -> 124 -> 48 -> 18 -> 7
9 -> 42 -> 39 -> 7
10 -> 112 -> 31 -> 18 -> 7
11 -> 111 -> 7
12 -> 104 -> 112 -> 31 -> 18 -> 7
13 -> 71 -> 125 -> 39 -> 7
14 -> 65 -> 92 -> 105 -> 111 -> 7
15 -> 27 -> 39 -> 7
16 -> 6 -> 125 -> 39 -> 7
17 -> 26 -> 119 -> 39 -> 7
18 -> 7
19 -> 119 -> 39 -> 7
20 -> 125 -> 39 -> 7
21 -> 51 -> 0 -> 24 -> 39 -> 7
22 -> 48 -> 18 -> 7
23 -> 62 -> 18 -> 7
24 -> 39 -> 7
25 -> 57 -> 31 -> 18 -> 7
26 -> 119 -> 39 -> 7
27 -> 39 -> 7
28 -> 40 -> 123 -> 7
29 -> 6 -> 125 -> 39 -> 7
30 -> 105 -> 111 -> 7
31 -> 18 -> 7
32 -> 3 -> 62 -> 18 -> 7
33 -> 24 -> 39 -> 7
34 -> 122 -> 18 -> 7
35 -> 114 -> 57 -> 31 -> 18 -> 7
36 -> 22 -> 48 -> 18 -> 7
37 -> 9 -> 42 -> 39 -> 7
38 -> 119 -> 39 -> 7
39 -> 7
40 -

> Answer: [+] Challenge 4/10. Generating random graph...
[+] Encrypting graph...
DESTINATION: 19
Graph shortest pathes:
0 -> 65 -> 67 -> 112 -> 88 -> 19
1 -> 30 -> 122 -> 120 -> 88 -> 19
2 -> 21 -> 107 -> 112 -> 88 -> 19
3 -> 7 -> 125 -> 19
4 -> 102 -> 19
5 -> 122 -> 120 -> 88 -> 19
6 -> 99 -> 88 -> 19
7 -> 125 -> 19
8 -> 7 -> 125 -> 19
9 -> 27 -> 106 -> 25 -> 102 -> 19
10 -> 75 -> 100 -> 19
11 -> 84 -> 6 -> 99 -> 88 -> 19
12 -> 31 -> 17 -> 34 -> 88 -> 19
13 -> 22 -> 77 -> 92 -> 88 -> 19
14 -> 31 -> 17 -> 34 -> 88 -> 19
15 -> 129 -> 92 -> 88 -> 19
16 -> 34 -> 88 -> 19
17 -> 34 -> 88 -> 19
18 -> 99 -> 88 -> 19
20 -> 102 -> 19
21 -> 107 -> 112 -> 88 -> 19
22 -> 77 -> 92 -> 88 -> 19
23 -> 47 -> 99 -> 88 -> 19
24 -> 18 -> 99 -> 88 -> 19
25 -> 102 -> 19
26 -> 42 -> 67 -> 112 -> 88 -> 19
27 -> 106 -> 25 -> 102 -> 19
28 -> 57 -> 47 -> 99 -> 88 -> 19
29 -> 89 -> 77 -> 92 -> 88 -> 19
30 -> 122 -> 120 -> 88 -> 19
31 -> 17 -> 34 -> 88 -> 19
32 -> 112 -> 88 -> 19
33 -> 10 -> 75 -> 100 -> 19
34 -> 

> Answer: [+] Challenge 7/10. Generating random graph...
[+] Encrypting graph...
DESTINATION: 5
Graph shortest pathes:
0 -> 99 -> 50 -> 69 -> 80 -> 58 -> 5
1 -> 91 -> 52 -> 80 -> 58 -> 5
2 -> 73 -> 69 -> 80 -> 58 -> 5
3 -> 17 -> 127 -> 97 -> 58 -> 5
4 -> 83 -> 15 -> 54 -> 80 -> 58 -> 5
6 -> 52 -> 80 -> 58 -> 5
7 -> 29 -> 17 -> 127 -> 97 -> 58 -> 5
8 -> 113 -> 50 -> 69 -> 80 -> 58 -> 5
9 -> 26 -> 107 -> 112 -> 80 -> 58 -> 5
10 -> 121 -> 97 -> 58 -> 5
11 -> 91 -> 52 -> 80 -> 58 -> 5
12 -> 68 -> 96 -> 97 -> 58 -> 5
13 -> 66 -> 20 -> 96 -> 97 -> 58 -> 5
14 -> 127 -> 97 -> 58 -> 5
15 -> 54 -> 80 -> 58 -> 5
16 -> 1 -> 91 -> 52 -> 80 -> 58 -> 5
17 -> 127 -> 97 -> 58 -> 5
18 -> 22 -> 54 -> 80 -> 58 -> 5
19 -> 50 -> 69 -> 80 -> 58 -> 5
20 -> 96 -> 97 -> 58 -> 5
21 -> 17 -> 127 -> 97 -> 58 -> 5
22 -> 54 -> 80 -> 58 -> 5
23 -> 67 -> 97 -> 58 -> 5
24 -> 121 -> 97 -> 58 -> 5
25 -> 38 -> 54 -> 80 -> 58 -> 5
26 -> 107 -> 112 -> 80 -> 58 -> 5
27 -> 67 -> 97 -> 58 -> 5
28 -> 97 -> 58 -> 5
29 -> 17 -> 1

> Answer: [+] Challenge 10/10. Generating random graph...
[+] Encrypting graph...
DESTINATION: 60
Graph shortest pathes:
0 -> 120 -> 99 -> 12 -> 60
1 -> 27 -> 44 -> 108 -> 60
2 -> 122 -> 99 -> 12 -> 60
3 -> 81 -> 128 -> 13 -> 39 -> 60
4 -> 50 -> 29 -> 39 -> 60
5 -> 98 -> 127 -> 95 -> 12 -> 60
6 -> 117 -> 99 -> 12 -> 60
7 -> 97 -> 11 -> 114 -> 60
8 -> 13 -> 39 -> 60
9 -> 31 -> 105 -> 95 -> 12 -> 60
10 -> 15 -> 22 -> 125 -> 11 -> 114 -> 60
11 -> 114 -> 60
12 -> 60
13 -> 39 -> 60
14 -> 113 -> 79 -> 29 -> 39 -> 60
15 -> 22 -> 125 -> 11 -> 114 -> 60
16 -> 79 -> 29 -> 39 -> 60
17 -> 125 -> 11 -> 114 -> 60
18 -> 63 -> 76 -> 53 -> 114 -> 60
19 -> 48 -> 44 -> 108 -> 60
20 -> 25 -> 108 -> 60
21 -> 49 -> 95 -> 12 -> 60
22 -> 125 -> 11 -> 114 -> 60
23 -> 30 -> 39 -> 60
24 -> 62 -> 27 -> 44 -> 108 -> 60
25 -> 108 -> 60
26 -> 84 -> 8 -> 13 -> 39 -> 60
27 -> 44 -> 108 -> 60
28 -> 126 -> 23 -> 30 -> 39 -> 60
29 -> 39 -> 60
30 -> 39 -> 60
31 -> 105 -> 95 -> 12 -> 60
32 -> 21 -> 49 -> 95 -> 12 -> 60
33 